# DAwHPC Practical 08 - Features, Trees, and Forests

In this practical you will investigate building Decision Trees, Pruning, and building Random Forests using `sklearn`.

## Setup

In [ ]:
import sklearn
import sklearn.tree
import sklearn.ensemble
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Practical

###  Part 1 - Introduction and Data Preparation

We’re going to use the Carseats dataset, which is provided as a CSV. This dataset contains 200 observations detailing the sales of child car seats across many shops.

Load and examine the variables in the dataset. You should see that it contains both continuous and categorical variables.

### Note: One-hot (dummy) Encoding

The sklearn tree classifiers don't currently deal with categorical features, so we need to use ["One-Hot Encoding"](https://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features) in order to transform a categorical feature of `n` levels into `n` new features. For each row, the new features will all be `0` except for the one feature representing the original value.

For example:

| ID | Rating |
| --- | --- |
| 1 | Low| 
| 2 | High|
| 3 | Medium |

would be one-hot encoded to:

| ID | Rating_Low | Rating_Medium | Rating_High |
| -- | ---------- | ------------- | ----------- |
| 1  | 1|0|0|
| 2 |0|0|1|
| 3|0|1|0|

### Data Preparation

Load and examine the variables in the dataset. You should see that it contains both continuous and categorical variables.

In [ ]:
carseats_df = pd.read_csv("DatasetsForPracticals/DecisionTreesAndForests/carseats.csv")

# One-hot-encode ShelveLoc
carseats_df = pd.get_dummies(carseats_df, columns=["ShelveLoc"])

# Map "Yes" and "No" to True/False
to_bool = {'Yes': True, 'No': False}
carseats_df["Urban"] = carseats_df["Urban"].map(to_bool)
carseats_df["US"] = carseats_df["US"].map(to_bool)

display(carseats_df.head())
display(carseats_df.dtypes)

The variables are defined as:
- `Sales`: Unit sales (in thousands) at each location
- `CompPrice`: Price charged by competitor at each location
- `Income`: Community income level (in thousands of dollars)
- `Advertising`: Local advertising budget for company at each location (in thousands of dollars)
- `Population`: Population size in region (in thousands)
- `Price`: Price company charges for car seats at each site
- `ShelveLoc`: A factor with levels Bad, Good and Medium indicating the quality of the shelving location for the car seats at each site
- `Age`: Average age of the local population
- `Education`: Education level at each location
- `Urban`: A factor with levels No and Yes to indicate whether the store is in an urban or rural location
- `US`: A factor with levels No and Yes to indicate whether the store is in the US or not

We want to be able to determine which features are the best predictors for high or low sales at a given location. Before we build a tree which will do this for us though, think about which features could be strong predictors of high or low sales at a particular location. Will any features have little correlation with the sales value?

Write-down your initial guesses so you can compare later.

Before we can train a classification tree using our data, we need to define what value of the Sales variable constitutes “high” or “low”. For this exercise, let’s define high sales as being above `8`. Create this variable and append it to the Carseats dataframe.

In [ ]:
carseats_df["HighSales"] = carseats_df["Sales"] > 8
carseats_df["HighSales"].value_counts()

### Gini Index

In the lecture, we discussed that there are a few possible feature selection criteria which Decision Trees use to best split the remaining data points at each node. We worked-through an example using Entropy and Information Gain.

In this lab, we’re going to be using another method, called the _Gini Index_. This captures the probability of a
data point being incorrectly classified when it is randomly chosen. It is defined by

$$\Large G = \sum_{k=1}^K p_k(1-p_k)$$

where K is the set of possible classes, and p_k is the probability of the class being chosen.

Similarly to the definition of Entropy for a binary event, the Gini index is symmetric. It has value 0 at 0% or
100% probability, and 0.5 at 50%. See Figure 2 below.

<div align="center">
    <img src="img/gini.png"/>
    <figcaption>Figure 2</figcaption>
    <br/>
</div>

At each node the Gini index is calculated for each of the remaining available features, and the feature which
results in the lowest value is chosen as the feature to split on. This is much simpler than the Information Gain
approach, which selects the next feature based on the largest reduction to the initial entropy at the node.

Information Gain tends to end-up splitting data to find the maximum reduction to impurity, which can result in smaller trees than Gini. Gini indexes don’t include any logarithm terms though, which are relatively expensive to compute. This can be a factor when training on large datasets. In general, both approaches produce trees with similar accuracies within a few percent [2].

### Part 2 - Classification Tree

We’re now ready to use the tree function to build and fit a classification tree to our data. As discussed, the Decision Tree algorithm will automatically pick the best features at each node in order to best split the data.

We’ll therefore give it all the features except Sales. We’ll name this one `tree_simple`, which will be justified in the next section.

**Note**: We have used Python's [dictionary unpacking](https://reference.codeproject.com/python3/dictionaries/python-dictionary-unpack) syntax below to help pass and re-use the required function parameters.

In [ ]:
X = carseats_df.drop(columns=["Sales", "HighSales"])
y = carseats_df["HighSales"]

tree_params = {
    "criterion": "gini",
    "max_depth": 6,
    "random_state": 0,
}

tree_simple = sklearn.tree.DecisionTreeClassifier(**tree_params)
tree_simple.fit(X, y);

We can now examine the resulting tree to see which features it has used, and its error rates.


In [ ]:
print("Feature importances:")
tree_importances = pd.Series(tree_simple.feature_importances_, index=X.columns)
display(tree_importances.sort_values(ascending=False))

print()
print("Depth: ", tree_simple.get_depth())
print("Leaf nodes: ", tree_simple.get_n_leaves())

From the output, we can see:
- The features the tree actually used to categorise the data, ranked by importance
- The number of terminal or “leaf” nodes - 35 here

Compare the features which the tree selected with your initial guesses from Part 1.

We can also plot the tree to visualise it:

In [ ]:
plt.figure()
sklearn.tree.plot_tree(tree_simple, max_depth=2, filled=True, proportion=True, fontsize=10, feature_names=X.columns)
plt.title("Carseats Decision Tree (Simple)")
plt.show()

From the plot, you should see that the tree has calculated `ShelveLoc_Good` as the most important feature to start with when predicing `Sales`, followed by `Price` in each child node. This matches the summary output above. You can change the `max_depth` setting in `plot_tree` to show more tree levels. Also note that the tree is quite high and has many leaves. This could indicate overfittig, which we will address in Part 3.

Let’s use the predict function to calculate predictions for the whole dataset, and create a confusion matrix to enable calculation of the accuracy.

In [ ]:
pd.crosstab(
    tree_simple.predict(X),
    y,
    margins=True,
    rownames=["Predicted"],
    colnames=["Actual"],
)

In [ ]:
print(f"Accuracy: {tree_simple.score(X, y)*100:.2f}%")

The approach above isn’t really valid though, since we want to predict our tree’s performance on unseen data, and use that as a fair measure.

To solve this, we need to split our data into training and testing datasets, then only train on the training subset, and measure the resulting prediction performance against the unseen testing data.

**Exercise**
1. Construct `carseats_train` and `carseats_test` dataframes randomly from the full dataset, choosing an 80/20 split
1. Train a new decision tree, `tree_split`, using only the training dataset
1. Evaluate the new tree, this time using the carseats_testing dataset. Does the "new" accuracy value surprise you? Consider how the calculated value has been affected by splitting our data into training and testing.

### Part 3 - Pruning

Let’s see if we can boost our tree’s accuracy. As discussed in the lecture, it’s possible that our tree may contain too many branches, and therefore be overfitted to the training data. This occurs when the tree algorithm continues to partition the data until the leaves contain a 
very small number of values for only one class. I.e. the tree has learned how to classify the training data and has become overfitted. We can try to prune our tree to see if our accuracy improves on the unseen testing data.

We're going to use ["Minimal Cost-Complexity Pruning"](https://scikit-learn.org/stable/modules/tree.html#minimal-cost-complexity-pruning), and determine how pruning has an effect on our classifier.

First, run a cost-complexity pruning to return the effective alphas used during pruning. We can also visualise how the tree impurity is affected by the pruning

In [ ]:
tree_split = sklearn.tree.DecisionTreeClassifier(**tree_params)
path = tree_split.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

plt.plot(ccp_alphas, impurities)
plt.xlabel("effective alpha")
plt.ylabel("total impurity of leaves")

Now we can feed the effective alpha values into new classifiers, and determine the value which gives the highest accuracy.

In [ ]:
trees = []
for ccp_alpha in ccp_alphas:
    tree_split = sklearn.tree.DecisionTreeClassifier(**tree_params, ccp_alpha=ccp_alpha)
    tree_split.fit(X_train, y_train)
    trees.append(tree_split)

acc_scores = [sklearn.metrics.accuracy_score(y_test, tree.predict(X_test)) for tree in trees]

tree_depths = [tree.tree_.max_depth for tree in trees]
plt.figure(figsize=(10,  6))
plt.grid()
plt.plot(ccp_alphas[:-1], acc_scores[:-1])
plt.xlabel("effective alpha")
plt.ylabel("Accuracy scores")
plt.show()

We can then use the best effective alpha value to train an optimally-pruned tree.

In [ ]:
ccp_alpha_best = ccp_alphas[np.argmax(acc_scores)]

tree_split = sklearn.tree.DecisionTreeClassifier(**tree_params, ccp_alpha=ccp_alpha_best)
tree_split.fit(X_train, y_train)

print("Depth: ", tree_split.get_depth())
print("Leaf nodes: ", tree_split.get_n_leaves())
print(f"Accuracy: {tree_split.score(X_test, y_test)*100:.2f}%")

Note the improved accuracy value, and the reduction in leaf nodes.

You can read more about this type of complexity pruning [here](https://scikit-learn.org/stable/auto_examples/tree/plot_cost_complexity_pruning.html).

### Part 4 - Bagging and Random Forests

As discussed in the lecture, we can collect many decision trees into an ensemble by bootstrapping the training dataset, then aggregating their predictions into a single result. This allows our ensemble model to generalise better than a single tree and prevents overfitting, since each tree only sees a subset of the training data. This approach is called _Bagging_.

One further method used when training decision trees for an ensemble algorithm is to only allow a subset of the available features to be used for splitting at each node of each tree. This means that at each node, a tree only gets to choose from a random set of features to split with. The reasoning for this is if one particularfeature ends-up being a very strong predictor, then it will be chosen much more regularly across the whole ensemble. If this feature is not allowed to participate in some random subset of trees, then we will end up with a more diverse ensemble. When this method is used, our ensemble is called a Random Forest.

The notation for this is that out of a total number of features (or predictors) $p$, we choose some integer value $m$ where $m <= p$. $m$ is a hyperparameter, i.e. one that we choose before training. Typically we choose $m = round(sqrt(p))$. Bagging is the special-case of a Random Forest where $m = p$. We can therefore tweak the value of $m$ (the `max_features` parameter in sklearn) to create either a Bagged ensemble, or a full Random Forest.

Let’s try training a bagged ensemble first, where $m = p$.

In [ ]:
forest_params = {
    **tree_params,
    "oob_score": True,
    "n_estimators": 100, 
    "max_features": len(X_train.columns),
}

forest = sklearn.ensemble.RandomForestClassifier(**forest_params)
forest.fit(X_train, y_train)

print(f"OOB Error: {(1-forest.oob_score_)*100:.2f}%")

From above, you should see:
- We specified the number of trees `n_estimators` to 100, which is the default. Larger values will take longer to train (measure it to see). As we discussed in the lecture, its possible to parallelise this training in order to address the scaling when training larger ensembles. This can be set with `n_jobs`
- The bag has an associated OOB error rate of 20.31%. Since each tree only receives a bootstrapped subset of the total training data (the "in-bag" data), we can use all the data points it was _not_ trained with to get a measure of its accuracy. The Out-of-Bag (OOB) error is calculated by doing this for every tree in the ensemble and averaging the result.

Similar to what we did previously, we can calculate the misclassification error rate by asking the ensemble to predict values for the testing set. Since we have many trees which each contribute a vote, the `predict` function automatically calcluates a majority result across the ensemble.

In [ ]:
display(pd.crosstab(
    forest.predict(X_test),
    y_test,
    margins=True,
    rownames=["Predicted"],
    colnames=["Actual"],
))

print(f"Accuracy: {forest.score(X_test, y_test)*100:.2f}%")

Now lets set $m = sqrt(p)$ (the default) and see the effect on the accuracy.

In [ ]:
forest_params["max_features"] = "sqrt"
forest = sklearn.ensemble.RandomForestClassifier(**forest_params)
forest.fit(X_train, y_train)

print(f"OOB Error: {(1-forest.oob_score_)*100:.2f}%")
display(pd.crosstab(
    forest.predict(X_test),
    y_test,
    margins=True,
    rownames=["Predicted"],
    colnames=["Actual"],
))
print(f"Accuracy: {forest.score(X_test, y_test)*100:.2f}%")

You can see that the OOB error has dropped slightly, and the accuracy has improved to 81%.

One final thing we can investigate is the relative feature importance of the Random Forest. With a single Decision tree, we can easily interpret it to see which variables have the largest impact on the output. This is not feasible with ensemble methods though. Instead, we can calculate the variable importances for the ensemble using ["Mean decrease in impurity (MDI)"](https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html#feature-importance-based-on-mean-decrease-in-impurity). This is a representation of the relative predictive power of each feature, based on what affect they have on reducing the impurity during training.

In [ ]:
forest_importances = pd.Series(forest.feature_importances_, index=X.columns)
display(forest_importances.sort_values(ascending=False))
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)

fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

Compare the relative variable importance to the plot of our Decision Tree. Do the same variables appear
near the top in both cases?

**Next steps**

If time permits:
- Tweak the value of `n_estimators` in the forest and measure the effect on the final accuracy
- Train more classifiers like above using another dataset, e.g., the breast cancer dataset (`sklearn.datasets.load_breast_cancer()`)


## References

1. This lab is based on a practical from An Introduction to Statistical Learning
1. [Raileanu and Stoffel, 2004 Theoretical comparison between the Gini Index and Information Gain criteria](https://www.unine.ch/files/live/sites/imi/files/shared/documents/papers/Gini_index_fulltext.pdf)